In [78]:
import os
import sys
import tensorflow as tf
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing

from andi_funcs import TrackGeneratorClassification, import_tracks, import_labels, package_tracks
from models import classification_model_2d
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
import numpy as np

tf.config.set_soft_device_placement(True)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
# Load validation data

train_x = np.load('./dataset/tracks.npy')
train_y = np.load('labels.npy')
shuffle_indices = np.random.permutation(int(len(train_y)))



Num GPUs Available:  1


In [45]:
train_x_shuffled = train_x[shuffle_indices]
train_y_shuffled = train_y[shuffle_indices]

indices_train = np.random.permutation(int(len(train_y)*0.8))
indices_val = np.random.permutation(int(len(train_y)*0.15)) + indices_train.max() + 1
indices_test = np.random.permutation(int(len(train_y)*0.05)) + indices_val.max() + 1

In [64]:
train_x = train_x_shuffled[indices_train]
train_y = train_y_shuffled[indices_train]
val_x = train_x_shuffled[indices_val]
val_y = train_y_shuffled[indices_val]

In [72]:
model = classification_model_2d()
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, None, 2)]    0                                            
__________________________________________________________________________________________________
tf.reverse_1 (TFOpLambda)       (None, None, 2)      0           input_3[0][0]                    
__________________________________________________________________________________________________
model_2 (Functional)            (None, None, 256)    241792      input_3[0][0]                    
                                                                 tf.reverse_1[0][0]               
__________________________________________________________________________________________________
global_max_pooling1d_2 (GlobalM (None, 256)          0           model_2[0][0]              

In [73]:
history = model.fit(train_x_shuffled,train_y_shuffled, validation_data=(val_x, val_y),
          use_multiprocessing=True, workers=8)


938/938 [==============================] - 152s 155ms/step - loss: 0.1820 - accuracy: 0.9560 - val_loss: 0.0263 - val_accuracy: 0.9900


In [74]:
history.history['accuracy']

[0.9559666514396667]

In [3]:
model.save('./model/model.h5')

NameError: name 'model' is not defined

In [7]:
model.load_weights('./model/model.h5')

In [18]:
swimmers = pd.read_csv('./dataset/swimmers1um.csv')

col = swimmers.iloc[0, :].size
row = swimmers.iloc[:, 0].size
print('columns: %d \nrows: %d' %(col, row))

list_dataset = []

for i in range(1, int((col+1)/2)):
    X = [] # clear X
    X.append(preprocessing.minmax_scale(swimmers.iloc[:, i*2-1]))# X
    X.append(preprocessing.minmax_scale(swimmers.iloc[:, i*2]))# Y
    list_dataset.append(np.array(X).T)

x = np.array(list_dataset).reshape(5, 999, 2)


columns: 11 
rows: 999


In [174]:
result = model.predict(train_x_shuffled[indices_test])
y = result.round(0)

In [175]:
np.savetxt('result.txt', y)